In [47]:
import numpy as np
from scipy.linalg import sqrtm
from scipy.stats import ortho_group

def generate_rff_matrix(X, tau, num_features):
    n, d = X.shape
    
    # Generate random Fourier vectors
    omega = np.random.normal(size=(d, num_features)) / np.sqrt(tau)
    b = 2 * np.pi * np.random.rand(num_features)
    cos_component = np.cos(X @ omega + b)  # (n, num_features)
    sin_component = np.sin(X @ omega + b)  # (n, num_features)
    
    # Combine cos and sin components into the RFF matrix
    rff_matrix = np.empty((2*num_features, n))
    rff_matrix[:num_features, :] = cos_component.T
    rff_matrix[num_features:, :] = sin_component.T
    return rff_matrix

def efficient_j_dpp_sampling(W, J, F=None):
    n, _ = W.shape
    S = set()
    p = np.linalg.norm(W, axis=1)**2
    F = [] if F is None else F
    
    for _ in range(J):
        prob = p / np.sum(p)
        sn = np.random.choice(n, p=prob)
        S.add(sn)
        
        ysn = W[sn]
        if len(S) == 1:
            fn = ysn
        else:
            fn = ysn - np.sum([np.dot(fl, ysn) * fl for fl in F], axis=0)
        
        fn_norm = np.linalg.norm(fn)
        fn_normalized = fn / fn_norm if fn_norm != 0 else fn
        
        F.append(fn_normalized)
        for i in range(n):
            if i != sn:
                p[i] -= (np.dot(fn_normalized, W[i]))**2
    
    return list(S), F

def compute_dpp_sample(rff_matrix, num_samples):
    # Compute the Gram matrix ΨΨ^T
    gram_matrix = rff_matrix @ rff_matrix.T
    
    # Perform eigendecomposition of the Gram matrix
    eigenvalues, eigenvectors = np.linalg.eigh(gram_matrix)
    # Keep only positive eigenvalues and corresponding eigenvectors
    positive_indices = eigenvalues > 0
    positive_eigenvalues = eigenvalues[positive_indices]
    positive_eigenvectors = eigenvectors[:, positive_indices]
    
    # Normalize eigenvectors
    normalized_eigenvectors = positive_eigenvectors / np.sqrt(positive_eigenvalues)
    sampled_indices = []
    for _ in range(num_samples):
        probabilities = np.sum(normalized_eigenvectors**2, axis=1)
        chosen_index = np.random.choice(len(probabilities), p=probabilities)
        sampled_indices.append(chosen_index)
        
        # Update eigenvectors using the projection trick
        projection_matrix = np.eye(len(normalized_eigenvectors)) - np.outer(normalized_eigenvectors[:, chosen_index], normalized_eigenvectors[:, chosen_index])
        normalized_eigenvectors = np.dot(projection_matrix, normalized_eigenvectors)
    
    return sampled_indices

    # return sampled_indices

def compute_weights(rff_matrix, sampled_indices):
    # Compute marginal probabilities for sampled indices
    L = rff_matrix.T @ rff_matrix
    print(sampled_indices[0])
    L_sampled = L[np.ix_(sampled_indices[0], sampled_indices[0])]
    print(1)
    pi = np.linalg.det(L_sampled)
    print(pi)
    
    # Compute weights
    weights = 1 / pi
    weights /= np.sum(weights)  # Normalize weights
    
    return weights

def dpp_coreset(X, tau, num_samples):
    r = max(10, 2 * num_samples)  # Choose r as a function of m (number of samples)
    rff_matrix = generate_rff_matrix(X, tau, r)
    
    sampled_indices = compute_dpp_sample(rff_matrix, num_samples)
    weights = compute_weights(rff_matrix, sampled_indices)
    
    return X[sampled_indices[0]], weights

# Example usage:
# Assuming X is your dataset (n x d numpy array), tau is the Gaussian kernel parameter, and m is the number of samples
X = np.random.randn(100, 2)  # Example dataset
tau = 1.0
m = 10
sampled_points, weights = dpp_coreset(X, tau, m)
print("Sampled Points:", sampled_points)
print("Weights:", weights)


(40,) (40, 40)


ValueError: probabilities do not sum to 1